In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch torchvision transformers

     |████████████████████████████████| 2.1MB 23.9MB/s 
     |████████████████████████████████| 901kB 28.0MB/s 
     |████████████████████████████████| 3.3MB 43.4MB/s 


In [ ]:
import json
import string
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import precision_recall_fscore_support
import time
from datetime import datetime
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertModel
from transformers import BertTokenizer
from transformers import AdamW
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

### loading the data

In [ ]:
path = 'drive/MyDrive/Colab/'
covid_data = [json.loads(event) for event in open(path+'project-data/covid.data.jsonl', "r").readlines()]

### run the best task 1 model on covid dataset

In [ ]:
def to_date(date_str):
    return datetime.strftime(datetime.strptime(date_str,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')

covid_data_sort = [sorted(event, key=lambda x : to_date(x["created_at"])) for event in covid_data]

In [ ]:
# remove url
def remove_urls(vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

def remove_ats(vTEXT):
    vTEXT = re.sub(r'@[^\s]* ', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

def extract_info(data, info="text"):
    res = []
    for i in range(len(data)):
        event = data[i]
        event_info = []
        for tw in event:
#             event_info.append(remove_ats(tw[info]))
            event_info.append(remove_ats(remove_urls(tw[info])))
        res.append(event_info)
    return res

covid_sents = extract_info(covid_data_sort)     # {event}  where event={source,apply1,apply2,...}

In [ ]:
def combine_replies(replies):
    res = ""
    for r in replies:
        res += r
    return res

covid_all = [[event[0], combine_replies(event[1:])] for event in covid_sents]

In [ ]:
# define the dataset class
class TwitrerDataset(Dataset):
    def __init__(self, X, source_maxlen, reply_maxlen):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.source_maxlen = source_maxlen
        self.reply_maxlen = reply_maxlen
        self.X = X
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        # selecting the sentence at the specific index
        source, replies = self.X[index]

        # preprocessing the text to be suitable for BERT
        s_tokens = self.tokenizer.tokenize(source)
        s_tokens = ['[CLS]'] + s_tokens + ['[SEP']       # insert CLS and SEP token
        if len(s_tokens) < self.source_maxlen:                # keep the same length of each sentence
            s_tokens = s_tokens + ['[PAD]' for _ in range(self.source_maxlen-len(s_tokens))]
        else:
            s_tokens = s_tokens[:self.source_maxlen-1] + ['SEP']

        r_tokens = self.tokenizer.tokenize(replies)
        r_tokens = r_tokens + ['[SEP']
        if len(r_tokens) < self.reply_maxlen:                # keep the same length of each sentence
            r_tokens = r_tokens + ['[PAD]' for _ in range(self.reply_maxlen-len(r_tokens))]
        else:
            r_tokens = r_tokens[:self.reply_maxlen-1] + ['SEP']

        tokens_ids = self.tokenizer.convert_tokens_to_ids(s_tokens) + self.tokenizer.convert_tokens_to_ids(r_tokens) # obtaining the indices of tokens in vocab
        tokens_ids_tensor = torch.tensor(tokens_ids)   

        attn_mask = (tokens_ids_tensor != 0).long()        # attention mask (identity where is padded)

        token_type_ids = torch.tensor([0 for _ in range(source_maxlen)]+[1 for _ in range(reply_maxlen)])

        return tokens_ids_tensor, attn_mask, token_type_ids     

In [ ]:
# hyperparameters
batch_size = 32
num_worders = 2
lr = 2e-5
source_maxlen, reply_maxlen = 30, 50

# creating instances of training and dev set
covid_set = TwitrerDataset(covid_all, source_maxlen=source_maxlen, reply_maxlen=reply_maxlen)

# creating dataset loader
covid_loader = DataLoader(covid_set, batch_size=batch_size, num_workers=num_worders)

print("Done preprocessing training and development data.")


Done preprocessing training and development data.


In [ ]:
class RumourClassifier(nn.Module):
    def __init__(self):
        super(RumourClassifier, self).__init__()
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        self.cls_layer = nn.Linear(768, 1)
        self.dropout = nn.Dropout(0.9)

    def forward(self, seq, attn_masks, token_type_ids):
        '''
        Inputs:
          -seq: Tensor of shape [B, T] containing token ids of sequences
          -attn_masks: Tensor of shape [B, T] containing attention masks to be used
        '''
        # feed the input to bert model to obtain contextualized representation
        outputs = self.bert_layer(seq, attention_mask=attn_masks, token_type_ids=token_type_ids)
        cont_reps = outputs.last_hidden_state

        # obtaining the representation of [CLS] head
        cls_rep = cont_reps[:, 0]

        # pooled_output = self.dropout(cls_rep)

        # feeding cls_rep into the classifier layer
        logits = self.cls_layer(cls_rep)

        return logits

In [ ]:
path = 'drive/MyDrive/Colab/'
gpu = 0

bert_net = RumourClassifier()
bert_net.load_state_dict(torch.load(path+"models/adamw_50_date.dat"))
bert_net.cuda(gpu)
print("loading the model: Done!")


loading the model: Done!


In [ ]:
def predict(net, dataloader, gpu):
    net.eval()
    preds = []
    count = 0
    with torch.no_grad():
        for seq, attn_masks, token_type_ids in dataloader:
            if count % 100 == 0:
                print(count)
            count += 1
            seq, attn_masks, token_type_ids = seq.cuda(gpu), attn_masks.cuda(gpu), token_type_ids.cuda(gpu)
            logits = net(seq, attn_masks, token_type_ids)
            soft_probs_cpu = (logits > 0).long().cpu().view(-1)
            preds.append(soft_probs_cpu)
    preds = torch.cat(preds)
    return preds

In [ ]:
gpu = 0

bert_preds = predict(bert_net, covid_loader, gpu)

0
100
200
300
400
500


In [ ]:
num_rumours = sum([1 for label in bert_preds if label==1])
num_nonrumours = sum([1 for label in bert_preds if label==0])

In [ ]:
print("There are {} instances(events) in covid dataset".format(len(covid_data)))
print("My bert network classifies {} rumours and {} non-rumours".format(num_rumours, num_nonrumours))

There are 17458 instances(events) in covid dataset
My bert network classifies 1268 rumours and 16190 non-rumours


### process and analyze the data

#### What are the popular hashtags of COVID-19 rumours and non-rumours? How much overlap or difference do they share?

In [ ]:
# for each event, get their hashtags and corresponding frequencies

hashtags = []                          
for event in covid_data:
    event_hashtags = []
    for tw in event:
        twitter_hashtags = tw['entities']['hashtags']
        for hashtag in twitter_hashtags:
            event_hashtags.append(hashtag['text'].lower())
    hashtags.append(Counter(event_hashtags))
    
hashtags[0]

Counter({'tenet': 1})

In [ ]:
rum_hashtags = []         # hashtags for rumour event(classified by my model)
nonrum_hashtags = []      # hashtags for non-rumour event(classified by my model)

for i in range(len(bert_preds)):
    if bert_preds[i] == 1:
        rum_hashtags.append(hashtags[i])
    elif bert_preds[i] == 0:
        nonrum_hashtags.append(hashtags[i])

In [ ]:
# combine all the hashtags for rumour and nonrumours respectively
combined_rum_hashtags = Counter()
for hts in rum_hashtags:
    combined_rum_hashtags += hts
combined_rum_hashtags = dict([(hs, f/num_rumours) for hs, f in sorted(combined_rum_hashtags.items(), key=lambda x: x[1], reverse=True)])

combined_nonrum_hashtags = Counter()
for hts in nonrum_hashtags:
    combined_nonrum_hashtags += hts
combined_nonrum_hashtags = dict([(hs, f/num_nonrumours) for hs, f in sorted(combined_nonrum_hashtags.items(), key=lambda x: x[1], reverse=True)])

In [ ]:
num_nonrumours, num_rumours

(16190, 1268)

In [ ]:
len(combined_nonrum_hashtags), len(combined_rum_hashtags)

(24591, 2257)

In [ ]:
top100_rum_hts = dict(list(combined_rum_hashtags.items())[:100])
top100_nonrum_hts = dict(list(combined_nonrum_hashtags.items())[:100])

# find the overlap of top100 of rumour hashtags and top100 of non-rumour hashtags, and the hashtags that only in rumours, and the hashtags that only in non-rumours
overlaps = []           # (hashtag, frequency in rumours, frequency in non-rumours)
only_rumours = []
only_nonrumours = []
for ht in top100_rum_hts:
    if ht in top100_nonrum_hts:
        overlaps.append((ht, top100_rum_hts[ht], top100_nonrum_hts[ht]))
    else:
        only_rumours.append((ht, top100_rum_hts[ht]))

for ht in top100_nonrum_hts:
   if ht not in top100_rum_hts:
       only_nonrumours.append((ht, top100_nonrum_hts[ht]))

print("There are {} hashtags both in rumours and non-rumours, {} hashtags only in rumours, {} hashtags only in non-rumours".format(len(overlaps), len(only_rumours), len(only_nonrumours)))

There are 68 hashtags both in rumours and non-rumours, 32 hashtags only in rumours, 32 hashtags only in non-rumours


In [ ]:
# average frequency in top 100 rumours (and top 100 non-rumours)
avg_freq_rum = np.mean(list(top100_rum_hts.values()))
avg_freq_nonrum = np.mean(list(top100_nonrum_hts.values()))

print("average frequency among top 100 frequentiest hashtags in rumours: {}".format(avg_freq_rum))
print("average frequency among top 100 frequentiest hashtags in non-rumours: {}".format(avg_freq_nonrum))

average frequency among top 100 frequentiest hashtags in rumours: 0.029952681388012624
average frequency among top 100 frequentiest hashtags in non-rumours: 0.05448486720197652


In [ ]:
import pandas as pd
pd.DataFrame(only_rumours[:20], columns=["hashtag", "rumour_freq"])

,hashtag,rumour_freq
0,breaking,0.029180
1,georgefloyd,0.022082
2,wuhan,0.014196
3,trumpandemic,0.012618
4,covid19ph,0.009464
5,riots2020,0.009464
6,firetrump,0.009464
7,dominiccummings,0.008675
8,india,0.007886
9,hoax,0.007886


In [ ]:
pd.DataFrame(only_nonrumours[:20], columns=["hashtag", "nonrumour_freq"])

NameError: ignored

In [ ]:
pd.DataFrame(overlaps[:20], columns=["hashtag", "rumour_freq", "nonrumour_freq"])

NameError: ignored

#### Do rumour source tweets convey a different sentiment/emotion to the non-rumour source tweets? What about their replies?

In [ ]:
!pip install   spacy
!pip install   spacytextblob
!python -m spacy download en_core_web_sm

In [ ]:
path = 'drive/MyDrive/Colab/'
bert_preds = torch.load(path + "models/preds.pt")

In [ ]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
text = 'it is  a good thing '
doc = nlp(text)
print(doc._.polarity)      # Polarity: -0.125
print(doc._.subjectivity)  # Sujectivity: 0.9
print(doc._.assessments)   # Assessments: [(['really', 'horrible'], -1.0, 1.0, None), (['worst', '!'], -1.0, 1.0, None), (['really', 'good'], 0.7, 0.6000000000000001, None), (['happy'], 0.8, 1.0, None)]

0.7
0.6000000000000001
[(['good'], 0.7, 0.6000000000000001, None)]


In [ ]:
def extract_info(data, info="text"):
    res = []
    for i in range(len(data)):
        event = data[i]
        event_info = []
        for tw in event:
            event_info.append(tw[info])
        res.append(event_info)
    return res

covid_sents = extract_info(covid_data)     # {event}  where event={source,apply1,apply2,...}

In [ ]:
def sentiment_marker(sent):
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe('spacytextblob')
    doc = nlp(sent)
    polarity = doc._.polarity     
    subjectivity = doc._.subjectivity 
    return polarity, subjectivity

In [ ]:
sentiment_sources = []                    # sentiment analysis on every comment(source+reply)
start = time.time()
for i in range(len(covid_sents)):
    if i % 10 == 0:
        print(i)
    event = covid_sents[i]
    sentiment = sentiment_marker(event[0])
    sentiment_sources.append(sentiment)

end = time.time()
end - start

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2